In [1]:
import os
import path
import time
import datetime

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
rc={'lines.linewidth': 2, 'axes.labelsize': 14, 'axes.titlesize': 14}
sns.set(rc=rc)
%matplotlib inline
from scipy import ndimage as ndi
import pandas as pd

import skimage.filters
import skimage.io
import skimage.morphology
import skimage.feature
import skimage.exposure

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

ImportError: No module named seaborn

In [12]:
def birth_time(path_to_file):
    stat = os.stat(path_to_file)
    btime = stat.st_birthtime
    raw_time = time.ctime(btime)
    creation_time = datetime.datetime.strptime(raw_time, "%a %b %d %H:%M:%S %Y")
    return creation_time

In [67]:
def colony_mask(fname, x1, x2, y1, y2):
    # Load the image
    im = skimage.io.imread(fname)
    
    # Make the structuring element
    selem = skimage.morphology.square(5)

    # Perform the median filter
    im_filt = skimage.filters.median(im, selem)

    # Clip out the colony of interest
    im_boxed = im_filt.copy()
    im_boxed = im_boxed[y1:y2,x1:x2]
    
    # Convert the uneven image to floating point
    im_float = im_boxed / 4096
    
    # Smooth to reduce noise
    im_smooth = skimage.filters.gaussian(im_float, 2)
    
    # Adjust exposure
    im_adj = skimage.exposure.equalize_adapthist(im_smooth)

    # Edge detection
    im_edge = skimage.filters.scharr(im_adj)
    im_edge = skimage.exposure.equalize_adapthist(im_edge)

    # Percentile edge intensity cutoff
    #cutoff = np.percentile(im_edge, 85)  

    # Threshhold based on scharr filter
    im_thr = (im_edge > 0.22)
 
    # Remove small objects and fill holes
    im_bw = skimage.morphology.remove_small_objects(im_thr, min_size=400)
    im_bw = ndi.morphology.binary_fill_holes(im_bw)

    '''
    # Get the histogram data
    hist_phase, bins_phase = skimage.exposure.histogram(im_edge)

    # Use matplotlib to make a pretty plot of histogram data
    fig, ax = plt.subplots(1, 1)
    ax.set_xlabel('pixel value')
    ax.set_ylabel('count')
    _ = ax.fill_between(bins_phase, hist_phase, alpha=0.75)
    
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im, cmap=plt.cm.viridis)      
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im_float, cmap=plt.cm.viridis)    
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im_adj, cmap=plt.cm.viridis)    
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im_edge, cmap=plt.cm.viridis)    
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im_thr, cmap=plt.cm.viridis)    
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im_bw, cmap=plt.cm.viridis)        
    '''

    # Edge detection
    im_edge2 = skimage.filters.scharr(im_bw)
    '''
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(im_edge2, cmap=plt.cm.viridis)       
    '''
    # Detect two radii
    hough_radii = np.arange(20, 200, 2)
    hough_res = hough_circle(im_edge2, hough_radii)
    
    # Select the best circle
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                               total_num_peaks=1)
    '''
    # Draw them
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    image = color.gray2rgb(im_adj)
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        image[circy, circx] = (220, 20, 20)

    ax.imshow(image, cmap=plt.cm.viridis)
    plt.show()
    '''
    
    # Generate a mask for the colony
    y_dif = y2 - y1
    x_dif = x2 - x1
    
    y, x = np.indices((y_dif, x_dif))
    mask_circle = (x - cx)**2 + (y - cy)**2 < radii**2
    
    '''
    fig, ax = plt.subplots(1, 1)
    with sns.axes_style('dark'):
        skimage.io.imshow(mask_circle, cmap=plt.cm.viridis)  
    '''

    return (mask_circle, im_float)

In [14]:
def average_over_colony(mask_circle, im_float):

    # Label binary image; backward kwarg says value in im_bw to consider backgr.
    im_labeled, n_labels = skimage.measure.label(
                                mask_circle, background=0, return_num=True)
    
    # Extract region props
    im_props = skimage.measure.regionprops(im_labeled, intensity_image=im_float)
    
    for prop in im_props:
        #print(prop.mean_intensity)
        colony_average = prop.mean_intensity
    
        return colony_average

In [78]:
data = np.empty((0,9))

# Image list 
for pos in ['Pos0','Pos1']:
    i = 0
    
    # Loop to obtain all file paths 
    for x in os.listdir(path="./test/Movie/"):
        if 'D'!= x[1]:
            sub_path = "./test/Movie/" + x + "/" + pos + "/" 
            a = np.sort([sub_path + y for y in os.listdir(path=sub_path) if 'img' == y[:3]])
            if i == 0:
                img_filenames = a
                i = 1
            else:    
                img_filenames = np.concatenate((img_filenames, a), axis = 0)        
    
    # Separating two channels
    fnames_rfp = img_filenames[np.arange(0,len(img_filenames),2)]
    fnames_gfp = img_filenames[np.arange(1,len(img_filenames),2)]
    
    # Isolate each colony and obtain colony average
    for fname in fnames_rfp:
        #Load the image
        #im = skimage.io.imread(fname)
        #fig, ax = plt.subplots(1, 1)
        #with sns.axes_style('dark'):
        #    skimage.io.imshow(im, cmap=plt.cm.viridis)  

        # Get creation time 
        creation_time = birth_time(fname)
        
        if 'Pos1' == pos:
            # First colony
            (mask_circle, im_float) = colony_mask(fname, 200, 400, 50, 250)
            colony_average1 = average_over_colony(mask_circle, im_float)
            
            # Third colony
            (mask_circle, im_float) = colony_mask(fname, 500, 700, 300, 500)
            colony_average3 = average_over_colony(mask_circle, im_float)            

            # Fifth colony
            (mask_circle, im_float) = colony_mask(fname, 800, 1000, 600, 800)
            colony_average5 = average_over_colony(mask_circle, im_float)     

        if 'Pos0' == pos:
            # First colony
            (mask_circle, im_float) = colony_mask(fname, 330, 590, 160, 420)
            colony_average1 = average_over_colony(mask_circle, im_float)
            
            # Third colony
            (mask_circle, im_float) = colony_mask(fname, 520, 780, 360, 610)
            colony_average3 = average_over_colony(mask_circle, im_float)            

            # Fifth colony
            (mask_circle, im_float) = colony_mask(fname, 700, 960, 520, 780)
            colony_average5 = average_over_colony(mask_circle, im_float)              
            
        # Create new entry
        image_entry = [fname, creation_time, pos, colony_average1, np.nan, colony_average3, np.nan, colony_average5, np.nan]
        data = np.append(data, [image_entry], axis=0)
        
    for fname in fnames_gfp:
        #Load the image
        #im = skimage.io.imread(fname)
        #fig, ax = plt.subplots(1, 1)
        #with sns.axes_style('dark'):
        #    skimage.io.imshow(im, cmap=plt.cm.viridis)  

        # Get creation time 
        creation_time = birth_time(fname)
        
        if 'Pos1' == pos:
            # Second colony
            (mask_circle, im_float) = colony_mask(fname, 320, 580, 150, 410)
            colony_average2 = average_over_colony(mask_circle, im_float)
            
            # Fourth colony
            (mask_circle, im_float) = colony_mask(fname, 620, 820, 430, 630)
            colony_average4 = average_over_colony(mask_circle, im_float)            

            # Sixth colony
            (mask_circle, im_float) = colony_mask(fname, 900, 1100, 750, 950)
            colony_average6 = average_over_colony(mask_circle, im_float)     

        if 'Pos0' == pos:
            # Second colony
            (mask_circle, im_float) = colony_mask(fname, 450, 650, 270, 460)
            colony_average2 = average_over_colony(mask_circle, im_float)
            
            # Fourth colony
            (mask_circle, im_float) = colony_mask(fname, 660, 830, 460, 640)
            colony_average4 = average_over_colony(mask_circle, im_float)            

            # Sixth colony
            (mask_circle, im_float) = colony_mask(fname, 840, 1030, 660, 830)
            colony_average6 = average_over_colony(mask_circle, im_float)                 
            
        # Create new entry
        image_entry = [fname, creation_time, pos, np.nan, colony_average2, np.nan, colony_average4, np.nan, colony_average6]
        data = np.append(data, [image_entry], axis=0)

# Convert into dataframe         
df = pd.DataFrame(data, columns=['File path', 'Creation time', 'Position', 'Col 1', 'Col 2', 'Col 3', 'Col 4', 'Col 5', 'Col 6'])
print(df)

/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/anaconda/lib/python3.6/site-packages/skimage/filters/rank/generic.py:102: UserWarning: Bitdepth of 11 may result in bad rank filter performance due to large number of bins.
  "performance due to large number of bins." % bitdepth)


                                           File path        Creation time  \
0  ./test/Movie/d1 11am - 430pm/Pos0/img_00000000...  2017-08-30 11:08:58   
1  ./test/Movie/d1 11am - 430pm/Pos0/img_00000000...  2017-08-30 11:14:03   
2  ./test/Movie/d3 8pm - d4 10am/Pos0/img_0000001...  2017-09-02 09:31:17   
3  ./test/Movie/d1 11am - 430pm/Pos0/img_00000000...  2017-08-30 11:08:58   
4  ./test/Movie/d1 11am - 430pm/Pos0/img_00000000...  2017-08-30 11:14:04   
5  ./test/Movie/d3 8pm - d4 10am/Pos0/img_0000001...  2017-09-02 09:31:18   
6  ./test/Movie/d1 11am - 430pm/Pos1/img_00000000...  2017-08-30 11:09:01   
7  ./test/Movie/d3 8pm - d4 10am/Pos1/img_0000001...  2017-09-02 09:31:59   
8  ./test/Movie/d1 11am - 430pm/Pos1/img_00000000...  2017-08-30 11:09:01   
9  ./test/Movie/d3 8pm - d4 10am/Pos1/img_0000001...  2017-09-02 09:31:59   

  Position      Col 1      Col 2      Col 3      Col 4      Col 5      Col 6  
0     Pos0  0.0806584        NaN  0.0785584        NaN  0.0835349        

In [ ]:
%reset